In [ ]:
import pymongo
import json
# Connect to your MongoDB instance
client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client["vaers"]
col = db["combinations"]

In [ ]:
with open('/home/sebastian/Documents/Masterarbeit/Combinations/All_Combinations_Filtered.json') as f:
    data = json.load(f)

In [ ]:

for item in data: 
    N = col.estimated_document_count()
    
    D_query = {
        "vax_data": {
            "$elemMatch": {
                "VAX_TYPE": item["vaccine"],
                "VAX_MANU": item["manufacturer"]
            }
        }
    }
    D = col.count_documents(D_query)
    
    E_query = {
        "symptoms": {
            "$in": item["symptoms"]
        }
    }
    E = col.count_documents(E_query)
    
    DE_query = {'$and': [D_query, E_query]}
    DE = col.count_documents(DE_query)
    
    
    De = D - DE
    dE = E - DE
    de = N - (DE + De + dE)
    entry = {
        "vaccine": item["vaccine"],
        "manufacturer": item["manufacturer"],
        "symptom": item["Event"],  # Assuming the symptom key is "Event" in your data
        "DE": DE,
        "dE": dE,
        "De": De,
        "de": de
    }
    
    # Insert the entry into the MongoDB collection
    col.insert_one(entry)
    

